<h1 align="center">AD-fbi: Automatic Differentiation Python Package</h1>

# Introduction

Our software package, *AD-fbi*, computes gradients using the technique of automatic differentiation. Automatic differentiation is important because it is able to solve the derivatives of complex functions at low cost while maintaining accuracy and stability. Its high practicality and precision make it applicable to a wide range of tasks, such as machine learning optimization, calculating financial loss or profit, and minimizing the cost of vaccine distribution. Automatic differentiation plays a crucial role in solving these problems. 

Prior to the technique of automatic differentiation, the two traditional computational techniques for solving differential equations are numerical differentiation and symbolic differentiation. The numerical differentiation method computes derivatives based on a finite numerical approximation which lacks stability and precision, and thus it is not ideal to be implemented in computer software. On the other hand, symbolic differentiation can become very computationally expensive as the function becomes more and more complex. 

The automatic differentiation technique transforms the entire computational process into a sequence of elementary arithmetic operations -- addition, subtraction, multiplication, division, and elementary functions. And then, it applies the chain rule on top of these elementary operations. In this way, it reduces the computational complexity that the model can reuse some of its parts computed in previous steps multiple times without re-calculating them. It keeps the same level of accuracy and precision as using symbolic differentiation. 

Our software package computes derivatives using the forward mode of auto differentiation and will later explore other extensions (different optimizer). 


# Background
* __Chain Rule__

Chain Rule is the most important concept in automatic differentiation. To solve the derivatives of a composite function, we use the Chain Rule to decompose each variable in the function into elementary components and mulptily them together. 

For a given function $f(y(x))$, the derivative of $f$ with respect to $ x $ is the following:

$$
\begin{align}
\frac{\partial f}{\partial x} = \frac{\partial f}{\partial y} \frac{\partial y}{\partial x}\\
\end{align}
$$

Since $y$ is a n-dimensional vector, we introduce the gradient operator $ \nabla $ into the expression to calculate the derivative of $y$ with respect to $x$, where $x = (x_1, ..., x_n)$:

$$
\begin{align}
\nabla y(x) =
\begin{bmatrix}
{\frac {\partial y}{\partial x_{1}}}(x)
\\
\vdots 
\\
{\frac {\partial y}{\partial x_{n}}}(x)
\end{bmatrix}
\end{align}
$$

The above expression is for a single $y$, but we typically have multiple $y$ in a neural network. Thus, for a given function $f(y_1(x), ..., y_n(x))$, the derivative of $f$ with respect to $x$ is defined as the following:

$$
\begin{align}
\nabla f_x = \sum_{i=1}^n \frac{\partial f}{\partial y_i} \nabla y_i(x)\\
\end{align}
$$


* __Forward Mode automatic differentiation__

Here we will give an example of how to do forward mode automatic differentiation.

Given x = $\begin{bmatrix} {x_1} \\ \vdots \\ {x_m} \end{bmatrix}$, where $k \in (1, 2, ..., m)$, we introduce the intermediate operations $ v $ to compute values at each elementary operation step.

For example, to compute the gradient $\nabla f$  of the function $f(x) = log(x_1) + sin(x_1 + x_2)$, the expression is derived as the following:

$\nabla f = \begin{bmatrix} \frac {\partial f} {\partial x_1} \\ \frac {\partial f} {\partial x_2} \end{bmatrix}  = \begin{bmatrix} \frac {1} {x_1} + \cos(x_1 + x_2) \\ \cos(x_1 + x_2) \end{bmatrix}$

The computation graph is shown here: 
![alt text](graph.png "Computational Graph")

$D_p v_{-1} = \nabla v_{-1}^T p = (\frac {\partial v_{-1}} {\partial x_1} \nabla x_{1})^T p = (\nabla x_{1})^T p = p_1$

$D_p v_{0} = \nabla v_{0}^T p = (\frac {\partial v_{0}} {\partial x_2} \nabla x_{2})^T p = (\nabla x_{2})^T p = p_2$

$D_p v_{1} = \nabla v_{1}^T p = (\frac {\partial v_{1}} {\partial v_{-1}} \nabla v_{-1} + \frac {\partial v_{1}}{\partial v_{0}} \nabla v_{0})^T p = (\nabla v_{-1} + \nabla v_0)^T p = D_p v_{-1} + D_p v_0$

$D_p v_{2} = \nabla v_{2}^T p = (\frac {\partial v_{2}} {\partial v_{1}} \nabla v_1)^T p = \cos(v_1) (\nabla v_1)^T p = \cos(v_1) D_p v_1$

$D_p v_{3} = \nabla v_{3}^T p = (\frac {\partial v_{3}} {\partial v_{-1}} \nabla v_{-1})^T p = \frac {1} {v_{-1}} (\nabla v_{-1})^T p = \frac {1} {v_{-1}} D_p v_{-1}$

$D_p v_{4} = \nabla v_{4}^T p = (\frac {\partial v_{4}} {\partial v_3} \nabla v_{3} + \frac {\partial v_{4}}{\partial v_{2}} \nabla v_{2})^T p = (\nabla v_{3} + \nabla v_2)^T p = D_p v_{3} + D_p v_2$

Thus, the final generalized formula is the following:

$$ D_p v_j = (\nabla v_j)^T p = (\sum_{i < j} \frac{\partial{v_j}} {\partial{v_i}} \nabla v_i)^T p = \sum_{i < j} \frac{\partial{v_j}} {\partial{v_i}} (\nabla v_i)^T p = \sum_{i < j} \frac{\partial{v_j}} {\partial{v_i}} D_p v_i$$ 


* __Jacobian Matrix__

Having derived the above system of equations, we want to use the Jacobian matrix to compute these derivatives systematically.

The Jacobian matrix is defined as the following:

$$
J_{p}(x_{1},x_{2}, ..., x_{n}) = \left[ \begin{matrix}
\frac{\partial y_{1}}{\partial x_{1}} & ... & \frac{\partial y_{1}}{\partial x_{n}} \\
\vdots  & \ddots & \vdots  \\
\frac{\partial y_{m}}{\partial x_{1}} & ... & \frac{\partial y_{m}}{\partial x_{n}}
\end{matrix} \right] 
$$


For example, a 2 by 2 Jacobian matrix with 2 variables looks like the following:

$$
J_{p}(x_{1},x_{2}) = \left[ \begin{matrix}
\frac{\partial y_{1}}{\partial x_{1}} & \frac{\partial y_{1}}{\partial x_{2}} \\
\frac{\partial y_{2}}{\partial x_{1}} & \frac{\partial y_{2}}{\partial x_{2}}
\end{matrix} \right] 
$$

We compute $J_{p}$ in the forward mode in the evaluation trace.

* __Seed Vector__

Seed vector $p$ provides an efficient way to retrieve elements in a given direction from the Jacobian matrix. For example, if we want to retreive the $i$, $j$ element from the Jacobian matrix, the seed vector $p = [\overrightarrow{i}, \overrightarrow{j}]$ helps to retrieve the element of $\frac{df_{i}}{dx_{j}}$. 

We will introduce the seed vector in the forward trace to facilitate the retrieval of any element in the Jacobian matrix to make the calculation process more efficient and faster. The default value of the seed vector is 1


# How to use *AD-fbi*

1. __Installing the package:__

   * install our package with the following line: 
    
    <code>python3 -m pip install AD-fbi </code> (NOT IMPLEMENTED)

   * clone our github repository with the following line: 

    <code>git clone https://code.harvard.edu/CS107/team06.git</code>

   * cd to team06 directory: 

    <code>cd team06</code>
  
   * create a virtual environment:

    <code>python3 -m venv fbi-env</code>

   * install dependencies from requirement.txt: 

    <code>python3 -m pip install -r requirement.txt</code> 

   * cd to source directory: 

    <code>cd src/</code>

   * open python console:
  
    <code>python3</code>


2. __Importing the package:__

   * import our package using python console:
    
    ```python
    from fbi.forward_mode import ForwardMode
    ```
    
3. __Instantiating AD objects:__

  3.1. Instantiate an AD object for a scalar function of a single input ($R$ -> $R$):
   * Input: 
    - <code>func</code>: function $f(x)$ 
    - <code>x</code>: a single value to be calculated with respect to $f(x)$ <code>func</code>
    - <code>seed</code>: seed vector
  
**Example:**
  * Define variables
  ```python
  func = lambda x: x + 1
  x, seed = 1, -1
  fm = ForwardMode(x, func, seed)
  ```

  * To calculate function value and dual value with respect to <code>func</code>:
   ```python
  print(fm.calculate_dual_number())
  ```
  output:
  ```
  >>> (2, -1)
  ```

  * To calculate $f(x)$:
   ```python
  print(fm.get_fx_value())
  ```
  output:
  ```
  >>> 2 
  ```

  * To calculate derivative with respect to $f(x)$:
   ```python
  print(fm.get_derivative())
  ```
  output:
  ```
  >>> -1 
  ```
 
3.2. (TO BE IMPLEMENTED) Instantiate an AD object for a vector function with multiple input variables ($R^m$ -> $R^n$): 

   ```python
   # input variable -  a vector of R^3
   x = np.array([0.5, 0.5, 0.2]) 

   # output function -  R^3 -> R^2
   f_x = lambda x, y, z: (x.log() + y + z, x * y + z)

   # instantiate an AD object
   fm = ad.ForwardMode(evaluate = x, function = f_x, seed=np.array([1,2,3]))
   ```

3.3. (TO BE IMPLEMENTED) Instantiate an AD object for a vector function with multiple input variables ($R^m$ -> $R$):

   ```python
   # input variable -  a vector of R^3
   x = np.array([0.5, 0.5, 0.2]) 

   # output function -  R^3 -> R
   f_x = lambda x, y, z: x * y + z

   # instantiate an AD object
   fm = ForwardMode(x, f_x, np.array([1,2,3]))
   ```

3.4. (TO BE IMPLEMENTED)  __Adam Optimizer:__
   We aim to develop optimizers such as the Adam optimizer for stochastic gradient descent to find the minima of a function. 
   Here is an example of how to interact with the feature.
   
   ```python
   from AD-fbi import optimization

   x = 1
   
   # define the function to find the minima points
   f_x = lambda x: x**3 + 3 * x

   # instantiate different optimizers
   opm_adam = Optimization.adam()
   opm_mom = Optimization.momentum()
   opm_ada = Optimization.ada()

   # find the mimina points and the running times for each optimizer
   # the optimizer methods will return the running time, the minimum values of the function (minima), and the locations to get the minima
   time_adam, min_adam, x_vals_adam = opm_adam(x, f_x, num_iter=100, epsilon=1e-10)
   time_mom, min_mom, x_vals_mom = opm_mom(x, f_x, num_iter=100, epsilon=1e-10)
   time_ada, min_ada, x_vals_ada = opm_ada(x, f_x, num_iter=100, epsilon=1e-10)

   ```


# Software Organization

__Folder structure__

The folder structure for the *AD-fbi* package:

```
team06/
      docs/
        graph.png
        directory_tree.png
        milestone1.ipynb
        milestone2_progress.md
        documentation.ipynb
      src/
        fbi/
          __init__.py
          dual_number.py
          forward_mode.py
        tests/
          __init__.py
          test_dual_number.py
          test_forward_mode.py
      requirements.txt
      setup.py (TO BE IMPLEMENTED)
      setup.cfg (TO BE IMPLEMENTED)
      README.md
      LICENSE
``` 

__Modules & Functionalities__

The AD-fbi package contains three essential modules: 1) a dual number module for elementary operations, 2) forward mode automatic differentiation, and 3) an Adam optimizer extension module. The followings are the decriptions of their functionalities:

   * dual_number.py: This module implements the DualNumber class, which contains methods to compute the function and derivative values of the elementary operations, and to overload the elementary operations for a dual number. These functions are essential to computing the primal and tangent trace in the forward mode of AD. Examples of elementary operations include: <code>+</code>, <code>-</code>, <code>\*</code>, <code>/</code>, <code>sin()</code>, <code>cos()</code>.
    
   * forward_mode.py: This module implements a ForwardModel class that provides a method to intialize a forward mode AD object, a method to construct a computational graph dictionary, and a method to run the forward mode process and returns the computed derivative of the function at the evaluated point.

   * optimization.py (extension module TO BE IMPLEMENTED): This module implements the Adam optimizer for stochastic gradient descent to facilitate the basic automatic differentiation functionalities. 

__Test Suite__

The test suite lives in the <code>team06/src/tests/</code> directory. It contains all pytests for our package. The YAML file under the <code>team06/.github/workflows/</code> directory defines the CI setup and jobs for GitHub Actions. When a new code is pushed to main branch, all tests are automatically trigered to test the code. 

__Package Distribution__

The package is distributed via PyPI. We first add a _pyproject.toml_ file to our project, then install `build` (a PEP517 package builder). After that, we build our package release and finally upload it to PyPI. We also add the setup.py and setup.cfg files to set up the backend of our package development. (TO BE IMPLEMENTED)

Users can install the package using the following command line:
```
python3 -m pip install AD-fbi
```
Install dependencies using the following command line:
```
python3 -m pip install -r requirements.txt
```
NOTE: Our package has not uploaded to PyPI at this point. You can clone our github repository and follow the instruction under the **How to use AD-fbi** section. 

__Package Dependencies__

We rely on the following external dependencies in our package:
* numpy==1.22.0
* pytest==7.1.2
* pytest_cov==4.0.0


# Implementation
    
* __Main Classes__
  * <code>DualNumbers</code>: class for operations with a dual number.
  * <code>ForwardMode</code>: class for forward mode differentiation.
  * <code>AdamOptimizer</code>(extension module TO BE IMPLEMENTED): class for the adam optimizer. This object has no attributes, but each method within the
   class requires a <code>x</code> input, a function <code>func</code> input, and the number of iterations <code>num_iter</code> for the specific optimizing method. Additionally, each method
   has their own optional hyperparameters which the user can input if they choose not to use our standard default values.

* __Core Data Structures & Dual Numbers__
  * Our primary core data structure is a numpy array, which we use to store both the variable list and the function list. Then using the
   methods within the `ForwardMode` class, we compute the jacobian and function value. We store the corresponding values or arrays in a tuple. For a single input, the jacobian and function value are singular values, and for multi-dimensional vector input, the stored values are arrays.


* __Classes Method & Name Attributes__
  * <code>DualNumbers</code>: 
    * A `__init__` method to initialize a <code>DualNumbers</code> object with a real number value and a dual number value.
    * A `__repr__` method to return the object representation in string format.
    * Multiple methods to overload the elementary operations for a dual number. e.g. `__add__`, `__sub__`, `__mul__`, `__div__`, `__pow__`, `__radd__`, `__rsub__`, `__rmul__`, `__rdiv__`, `__rpow__`, etc.
    * Multiple methods to compare dual numbers. e.g. `__ne__`, `__eq__`, etc.
    * Multiple methods to transform a dual number. e.g. `sqrt`, `log`, `sin`, `cos`, `exp`, `tan`, etc.

  * <code>ForwardMode</code>:
    * A `__init__` method  to initialize a `ForwardMode` Object with an input value `x`, a function `func`, and a seed vector `seed`.
    * A `get_fx_value` method to run the forward mode process and return a function value at the evaluated point `x`.
    * A `calculate_dual_number` method to run the forward mode process and return the evaluated value and the derivative of the input function at the evaluated point `x`.
    * A `get_derivative` method to run the forward mode process and return a value of directional derivative corresponding to a specific seed vector.

* __Graph Class__
  * We do not use a graph class to resemble the computational graph in forward mode. 
  
* __Dealing With Operator Overloading and Elementary Functions__

   * For the overloading operator template (like `__add__` for our special dual number class object), we implemented multiple methods such as `__radd__`, `__rsub__`, `__rmul__`, etc. to handle both cases of dual number + integer, and integer + dual number. The `__r*__` methods are necessary to handle overloading. 

   *  As listed above, within the <code>DualNumbers</code> class we’ve overloaded the simple arithmetic functions (addition, subtraction, multiplication,
   division, negation, power, equal, and not equal) to calculate both the value and the dual number. We’ve also defined our own
   elementary functions, such as `sin` and `sqrt` etc. to compute the value and the derivative. This module
   generalizes each of the functions in order for the <code>ForwardMode</code> class to handle both scalar and vector inputs. Each method has also implemented raise error attribute to deal with all possible types of invalid inputs. The output is a tuple of the function value and the derivative, which
   is used in the <code>ForwardMode</code> class.
   
* __Dealing With Operator Overloading on Reverse Mode__

   * We currently are not interested in implementing a reverse mode on our package.

* __Dealing With MultiDimensional Input and Output Function__

   * Use `try` and `except` to handle multi-dimensional and single-dimensional case separetly. For the multi-dimensional case, we design a helper function to loop through all of the functions' inputs and reassign the value/derivative as a vector. 
   * We treat functions as a list (so high dimensional functions will be a list of functions)
   * The `grad()` function (or jacobian function) is generic to both single-dimensional and multi-dimensional(as they are either a list of 1 or a list of mulitple functions).
   
* __External Dependencies__
   * We use the numpy library to create our data structure for the computational graph and perform
   computations outside of those we created in our dual_number class.

# License

Our *AD-fbi* package is licensed under the GNU General Public License v3.0. This free software license allows users to do just about anything they want with our project, except distribute closed source versions. This means that any improved versions of our package that individuals seek to release must also be free software. We find it essential to allow users to help each other share their bug fixes and improvements with other users. Our hope is that users of this package continually find ways to improve it and share these improvements within the broader scientific community that uses automatic differentation.